<a href="https://colab.research.google.com/github/AdityaMK15/AI/blob/main/A*_8_PUZZLE_lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import heapq

class PuzzleState:
    def __init__(self, board, zero_pos, moves=0, previous=None):
        self.board = board
        self.zero_pos = zero_pos
        self.moves = moves
        self.previous = previous
    def get_neighbors(self):
        neighbors = []
        x, y = self.zero_pos
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_board = [list(row) for row in self.board]
                new_board[x][y], new_board[nx][ny] = new_board[nx][ny], new_board[x][y]
                neighbors.append(PuzzleState(new_board, (nx, ny), self.moves + 1, self))

        return neighbors

    def is_goal(self, goal):
        return self.board == goal

    def heuristic(self):
        distance = 0
        for i in range(3):
            for j in range(3):
                value = self.board[i][j]
                if value != 0:
                    target_x = (value - 1) // 3
                    target_y = (value - 1) % 3
                    distance += abs(i - target_x) + abs(j - target_y)
        return distance

    def __lt__(self, other):
        return (self.moves + self.heuristic()) < (other.moves + other.heuristic())

    def show_possible_moves(self):
        x, y = self.zero_pos
        possible_moves = []
        directions = {
            "Up": (x - 1, y),
            "Down": (x + 1, y),
            "Left": (x, y - 1),
            "Right": (x, y + 1)
        }
        for direction, (nx, ny) in directions.items():
            if 0 <= nx < 3 and 0 <= ny < 3:
                possible_moves.append(direction)
        return possible_moves

def a_star(initial_board, goal_board):
    initial_zero_pos = [(i, row.index(0)) for i, row in enumerate(initial_board) if 0 in row][0]
    start_state = PuzzleState(initial_board, initial_zero_pos)

    open_set = []
    heapq.heappush(open_set, start_state)
    closed_set = set()

    while open_set:
        current_state = heapq.heappop(open_set)

        if current_state.is_goal(goal_board):
            return reconstruct_path(current_state)

        closed_set.add(tuple(map(tuple, current_state.board)))

        for neighbor in current_state.get_neighbors():
            neighbor_tuple = tuple(map(tuple, neighbor.board))

            if neighbor_tuple in closed_set:
                continue

            heapq.heappush(open_set, neighbor)

    return None
def reconstruct_path(state):
    path = []
    while state:
        path.append(state.board)
        state = state.previous
    return path[::-1]

def get_board_input(prompt):
    while True:
        try:
            board = []
            print(prompt)
            for i in range(3):
                row = list(map(int, input(f"Enter row {i + 1} (3 numbers, separated by spaces): ").split()))
                if len(row) != 3 or any(num < 0 or num > 8 for num in row):
                    raise ValueError
                board.append(row)
            if sorted(num for row in board for num in row) == list(range(9)):
                return board
            else:
                print("The board must contain all numbers from 0 to 8 exactly once.")
        except ValueError:
            print("Invalid input. Please enter 3 numbers (0-8) for each row.")


if __name__ == "__main__":
    initial_board = get_board_input("Enter the start state:")
    goal_board = get_board_input("Enter the goal state:")

    start_state = PuzzleState(initial_board, [(i, row.index(0)) for i, row in enumerate(initial_board) if 0 in row][0])
    possible_moves = start_state.show_possible_moves()

    print(f"\nPossible moves from the current position of zero: {possible_moves}\n")

    solution = a_star(initial_board, goal_board)

    if solution:
        print("Solution steps:")
        for step in solution:
            for row in step:
                print(row)
            print()
    else:
        print("No solution found.")


Enter the start state:
Enter row 1 (3 numbers, separated by spaces): 2 8 3
Enter row 2 (3 numbers, separated by spaces): 1 6 4
Enter row 3 (3 numbers, separated by spaces): 0 7 5
Enter the goal state:
Enter row 1 (3 numbers, separated by spaces): 1 2 3
Enter row 2 (3 numbers, separated by spaces): 8 0 4
Enter row 3 (3 numbers, separated by spaces): 7 6 5

Possible moves from the current position of zero: ['Up', 'Right']

Solution steps:
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]

[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

[2, 8, 3]
[1, 0, 4]
[7, 6, 5]

[2, 0, 3]
[1, 8, 4]
[7, 6, 5]

[0, 2, 3]
[1, 8, 4]
[7, 6, 5]

[1, 2, 3]
[0, 8, 4]
[7, 6, 5]

[1, 2, 3]
[8, 0, 4]
[7, 6, 5]

